In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'cities', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by City
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
pd.options.display.max_rows = None

In [9]:
df.rename(columns={'city_name': 'City'}, inplace=True)
df['City'] = df['City'].fillna('Not in City Limits')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,36,"6,262","31,882","146,859"
Arlington,188,"1,647","12,906","295,638"
Auburn,"4,291","64,305","154,967","1,673,887"
Bainbridge Island,20,244,"2,339","218,668"
Beaux Arts,0,0,0,453
Bellevue,"20,800","237,068","495,423","3,179,334"
Black Diamond,143,730,"1,172","50,115"
Bonney Lake,"1,834","17,699","31,239","212,627"
Bothell,"18,876","148,937","245,346","1,348,306"


In [10]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,3.4%,17.2%,79.4%
Arlington,0.1%,0.5%,4.2%,95.3%
Auburn,0.2%,3.4%,8.2%,88.2%
Bainbridge Island,0.0%,0.1%,1.1%,98.8%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.5%,6.0%,12.6%,80.8%
Black Diamond,0.3%,1.4%,2.2%,96.1%
Bonney Lake,0.7%,6.7%,11.9%,80.7%
Bothell,1.1%,8.5%,13.9%,76.5%


#### AM Peak (7 to 8 AM)

In [11]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0,"2,953","2,664","7,066"
Arlington,15,96,982,"20,832"
Auburn,378,"9,880","25,677","101,988"
Bainbridge Island,1,17,"1,205","15,118"
Beaux Arts,0,0,0,29
Bellevue,"1,416","37,161","64,381","188,163"
Black Diamond,23,71,266,"3,610"
Bonney Lake,330,"2,676","2,051","12,798"
Bothell,487,"28,961","25,703","64,329"


In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,23.3%,21.0%,55.7%
Arlington,0.1%,0.4%,4.5%,95.0%
Auburn,0.3%,7.2%,18.6%,73.9%
Bainbridge Island,0.0%,0.1%,7.4%,92.5%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.5%,12.8%,22.1%,64.6%
Black Diamond,0.6%,1.8%,6.7%,90.9%
Bonney Lake,1.8%,15.0%,11.5%,71.7%
Bothell,0.4%,24.2%,21.5%,53.8%


#### PM Peak (5 to 6 PM)

In [13]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,19,"3,236","2,546","7,981"
Arlington,13,142,"1,432","24,027"
Auburn,369,"10,835","16,314","127,319"
Bainbridge Island,1,26,103,"18,944"
Beaux Arts,0,0,0,35
Bellevue,"4,651","36,553","64,303","215,852"
Black Diamond,23,54,87,"4,301"
Bonney Lake,148,"2,157","4,051","14,660"
Bothell,"5,343","18,736","43,226","70,577"


In [14]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.1%,23.5%,18.5%,57.9%
Arlington,0.1%,0.6%,5.6%,93.8%
Auburn,0.2%,7.0%,10.5%,82.2%
Bainbridge Island,0.0%,0.1%,0.5%,99.3%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,1.4%,11.4%,20.0%,67.2%
Black Diamond,0.5%,1.2%,1.9%,96.3%
Bonney Lake,0.7%,10.3%,19.3%,69.8%
Bothell,3.9%,13.6%,31.4%,51.2%


# Network Delay
Totals scaled to match network totals. 

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
City,
Algona,735
Arlington,278
Auburn,"5,393"
Bainbridge Island,122
Beaux Arts,0
Bellevue,"18,074"
Black Diamond,37
Bonney Lake,"1,143"
Bothell,"11,565"


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [16]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
City,
Algona,625
Arlington,236
Auburn,"4,585"
Bainbridge Island,104
Beaux Arts,0
Bellevue,"15,367"
Black Diamond,31
Bonney Lake,972
Bothell,"9,832"
